In [98]:
import numpy as np
import matplotlib.pyplot as plt

# 1. Solucion a la ecuacion Ax=b

In [196]:
def swap_rows(A, i, j):
    """
    Intercambia las filas i y j de la matriz A.
    """
    A[[i, j]] = A[[j, i]]  # Intercambio sin necesidad de variable temporal.


A)

B)

C)

In [197]:
def fact_plu(A):
    """
    Factoriza la matriz A en las matrices P, L y U utilizando la factorización PLU.
    
    Parameters:
    -----------
    A : np.ndarray
        Matriz a factorizar.
    
    Returns:
    --------
    P, L, U : np.ndarray
        Matrices de permutación (P), triangular inferior (L) y triangular superior (U).
    """
    m, n = np.shape(A)
    if m != n:
        raise ValueError("La matriz debe ser cuadrada")
    
    P = np.eye(n)  # Matriz de permutación
    L = np.zeros((n, n))  # Matriz triangular inferior
    U = np.copy(A)  # Copia de la matriz original
    
    for j in range(n - 1):
        # Pivoteo parcial
        if np.abs(U[j, j]) == 0:
            max_row = np.argmax(np.abs(U[j:, j])) + j
            if U[max_row, j] == 0:
                raise ValueError("La matriz es singular")
            swap_rows(U, j, max_row)
            swap_rows(P, j, max_row)
        
        for i in range(j + 1, n):
            L[i, j] = U[i, j] / U[j, j]
            U[i] -= L[i, j] * U[j]
    
    L += np.eye(n)  # Añadir la diagonal de unos a L
    return P, L, U

def forward_substitution(L, b):
    """
    Resuelve el sistema Ly = b, donde L es una matriz triangular inferior.
    
    Parameters:
    -----------
    L : np.ndarray
        Matriz triangular inferior.
    b : np.ndarray
        Vector independiente.
    
    Returns:
    --------
    y : np.ndarray
        Solución del sistema Ly = b.
    """
    n = len(b)
    y = np.zeros_like(b)
    
    for i in range(n):
        y[i] = (b[i] - np.dot(L[i, :i], y[:i])) / L[i, i]
    
    return y

def backward_substitution(U, y):
    """
    Resuelve el sistema Ux = y, donde U es una matriz triangular superior.
    
    Parameters:
    -----------
    U : np.ndarray
        Matriz triangular superior.
    y : np.ndarray
        Vector independiente.
    
    Returns:
    --------
    x : np.ndarray
        Solución del sistema Ux = y.
    """
    n = len(y)
    x = np.zeros_like(y)
    
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i + 1:], x[i + 1:])) / U[i, i]
    
    return x

def solve_plu(A, b):
    """
    Resuelve el sistema Ax = b usando la factorización PLU.
    
    Parameters:
    -----------
    A : np.ndarray
        Matriz de coeficientes A.
    b : np.ndarray
        Vector de términos independientes b.
    
    Returns:
    --------
    x : np.ndarray
        Solución del sistema Ax = b.
    """
    # Factorización PLU
    P, L, U = fact_plu(A)
    
    # Resolver Ly = Pb
    Pb = np.dot(P, b)
    y = forward_substitution(L, Pb)
    
    # Resolver Ux = y
    x = backward_substitution(U, y)
    
    return x


D)

In [198]:
def fact_cholesky(A):
    """
    Realiza la factorización de Cholesky de una matriz A.
    
    Parámetros:
    A (numpy.ndarray): Matriz simétrica y definida positiva.
    
    Retorna:
    numpy.ndarray: Matriz L tal que A = LL^T.
    """
    n = A.shape[0]
    L = np.zeros_like(A)
    
    # Comprobar si la matriz es cuadrada
    if A.shape[0] != A.shape[1]:
        raise ValueError('La matriz no es cuadrada.')
    
    # Comprobar si la matriz es simétrica
    if not np.allclose(A, A.T):
        raise ValueError('La matriz no es simétrica.')
    
    for i in range(n):
        temp_sum = np.dot(L[i, :i], L[i, :i])
        diag_element = A[i, i] - temp_sum
        if diag_element <= 0:
            raise ValueError('La matriz no es definida positiva.')
        
        L[i, i] = np.sqrt(diag_element)
        
        for j in range(i + 1, n):
            temp_sum = np.dot(L[j, :i], L[i, :i])
            L[j, i] = (A[j, i] - temp_sum) / L[i, i]
    
    return L
def solve_cholesky(A, b):
    """
    Resuelve el sistema Ax = b utilizando la factorización de Cholesky A = LL^T.
    
    Parámetros:
    A (numpy.ndarray): Matriz simétrica y definida positiva.
    b (numpy.ndarray): Vector b.
    
    Retorna:
    numpy.ndarray: Vector solución x.
    """
    # Obtener la factorización de Cholesky A = LL^T
    L = fact_cholesky(A)
    # Resolver Ly = b utilizando sustitución hacia adelante
    y = forward_substitution(L, b)
    # Resolver L^T x = y utilizando sustitución hacia atrás
    x = backward_substitution(L.T, y)
    
    return x

In [199]:
A = np.random.rand(100, 100)
x = np.arange(1, 101, dtype=np.float64)  # Vector de 1 a 100
b = A @ x
x_teorico = np.array([i + 1 for i in range(100)])
print("Vector b:", b)

E), F), G) y H)

In [200]:
np.random.seed(42)
A = np.random.rand(100, 100)
x_original = np.arange(1, 101, dtype=np.float64)

# Crear la matriz N que es simétrica y definida positiva
N = A @ A.T
b_N = N @ x_original  # Calcular el vector b

# Solucionador PLU
x_plu = solve_plu(N, b_N)
print(" PLU:", x_plu)
print( np.allclose(N @ x_plu, b_N))

# Solucionador Cholesky (N simétrica y definida positiva)
x_cholesky = solve_cholesky(N, b_N)
print(" Cholesky:", x_cholesky)
print(np.allclose(N @ x_cholesky, b_N))

In [201]:
def calcular_errores(x_teorico, x_calculado):
    """
    Calcula el error absoluto y el error relativo entre la solución teórica y la solución calculada.
    
    Parámetros:
    x_teorico (numpy.ndarray): Vector solución teórica x_s.
    x_calculado (numpy.ndarray): Vector solución calculada x.
    
    Retorna:
    tuple: error absoluto, error relativo
    """
    error_absoluto = np.linalg.norm(x_teorico - x_calculado)
    error_relativo = error_absoluto / np.linalg.norm(x_teorico) if np.linalg.norm(x_teorico) != 0 else np.nan
    return error_absoluto, error_relativo

# Guardar los errores en listas
errores_abs_plu = []
errores_rel_plu = []
errores_abs_cholesky = []
errores_rel_cholesky = []

# Calcular errores para 30 iteraciones
for i in range(30):
    # Genera una matriz A aleatoria por cada iteración
    A = np.random.rand(100, 100)
    # Genera el vector teórico x
    x_original = np.arange(1, 101, dtype=np.float64)
    # Calcular para Cholesky
    N = A @ A.T
    b_N = N @ x_original
    # Solucionador PLU
    x_plu = solve_plu(N, b_N)
    err_abs_plu, err_rel_plu = calcular_errores(x_original, x_plu)
    errores_abs_plu.append(err_abs_plu)
    errores_rel_plu.append(err_rel_plu)
    # Solucionador Cholesky
    x_cholesky = solve_cholesky(N, b_N)
    err_abs_cholesky, err_rel_cholesky = calcular_errores(x_original, x_cholesky)
    errores_abs_cholesky.append(err_abs_cholesky)
    errores_rel_cholesky.append(err_rel_cholesky)

# Convertir listas a arrays para que sea más fácil de manipular...
errores_abs_plu = np.array(errores_abs_plu)
errores_rel_plu = np.array(errores_rel_plu)
errores_abs_cholesky = np.array(errores_abs_cholesky)
errores_rel_cholesky = np.array(errores_rel_cholesky)

# Graficar errores
plt.figure(figsize=(10, 6))

# Graficar errores absolutos
plt.subplot(2, 1, 1)
plt.plot(errores_abs_plu, label='Error absoluto PLU', marker='o')
plt.plot(errores_abs_cholesky, label='Error absoluto Cholesky', marker='x')
plt.title('Errores Absolutos')
plt.xlabel('Iteración')
plt.ylabel('Error absoluto')
plt.grid()
plt.legend()

# Graficar errores relativos
plt.subplot(2, 1, 2)
plt.plot(errores_rel_plu, label='Error relativo PLU', marker='o')
plt.plot(errores_rel_cholesky, label='Error relativo Cholesky', marker='x')
plt.title('Errores Relativos')
plt.xlabel('Iteración')
plt.ylabel('Error relativo')
plt.legend()

plt.grid()
plt.show()

# Calcular y mostrar promedios de errores
print(f"Error absoluto promedio PLU: {np.mean(errores_abs_plu):.6e}")
print(f"Error relativo promedio PLU: {np.mean(errores_rel_plu):.6e}")
print(f"Error absoluto promedio Cholesky: {np.mean(errores_abs_cholesky):.6e}")
print(f"Error relativo promedio Cholesky: {np.mean(errores_rel_cholesky):.6e}")


i)

### Análisis del Uso de Errores Absoluto y Relativo

En este contexto particular, donde las magnitudes de los valores de las matrices no varían significativamente, se puede argumentar que tanto el **error absoluto** como el **error relativo** ofrecen información similar sobre el rendimiento de los algoritmos. 

- **Error Absoluto**: Mide la diferencia total entre la solución teórica \( x_s \) y la solución calculada \( x \). Esto es útil para entender cuánto se desvía la solución calculada de la solución real.
  
- **Error Relativo**: Mide esa misma diferencia, pero la normaliza con respecto a la magnitud de la solución teórica, lo que permite ver el error en un contexto más proporcional.

Dado que se utilizan valores constantes para las matrices, el error relativo puede considerarse un escalamiento del error absoluto. Por lo tanto, en este caso, **ambos errores proporcionan información similar**, lo que implica que cualquiera de ellos podría ser adecuado para evaluar el funcionamiento de los algoritmos.

j)

In [202]:
print('Media (relative error) PLU:', err_rel_plu.mean())
print('Media (relative error) Cholesky:', err_rel_cholesky.mean())

In [203]:
print('Media (error absoluto) PLU:', err_abs_plu.mean())
print('Media (error absoluto) Cholesky:', err_abs_cholesky.mean())


#### 1. **Precisión**:

En conclusión, **el método Cholesky es más preciso** que el método PLU, ya que presenta menores valores en ambas métricas.

#### 2. **Exactitud**:
- La exactitud está relacionada con la cercanía de las soluciones calculadas respecto a la solución teórica. Al observar que el método Cholesky tiene menores errores absolutos y relativos, podemos afirmar que **Cholesky es más exacto** en este caso particular.
- **Elección del Error**: En este contexto, es más apropiado utilizar el **error absoluto** y el **error relativo**, ya que ambos reflejan de manera efectiva el rendimiento de los algoritmos. Sin embargo, en general, el **error relativo** puede ser más útil en contextos donde las magnitudes de las soluciones pueden variar considerablemente, ya que proporciona una perspectiva normalizada del rendimiento.

- **Conclusiones sobre los Algoritmos**:
  - **Chlesky**: Más preciso y exacto en este caso particular, con menores errores absolutos y relativos.
  - **PLU**: Aunque presenta un rendimiento aceptable, tiene un mayor error en comparación con Cholesky.


# Jacobi y Gauss


### (a) Algoritmo de Jacobi

In [204]:
def Mat_Jacobi(n):
    """Genera una matriz diagonal dominante aleatoria de tamaño n x n."""
    A = np.random.rand(n, n)
    for i in range(n):
        A[i][i] = np.sum(np.abs(A[i])) + 1  # Asegurar que la matriz sea diagonal dominante
    return A

In [205]:

def jacobi(A, b, x0=None, tol=1e-10, max_iter=100):
    """
    Resuelve el sistema Ax = b usando el método iterativo de Jacobi.

    Parámetros:
    A (numpy.ndarray): Matriz de coeficientes de tamaño n x n.
    b (numpy.ndarray): Vector de términos independientes.
    x0 (numpy.ndarray): Vector inicial de la solución, por defecto es un vector de ceros.
    tol (float): Tolerancia para el criterio de convergencia.
    max_iter (int): Número máximo de iteraciones.

    Retorna:
    numpy.ndarray: Aproximación de la solución x.
    """
    n = A.shape[0]
    # Inicializar con ceros si no se proporciona un vector inicial
    if x0 is None:
        x0 = np.zeros_like(b)
    x = x0.copy()

    # Iterar hasta el número máximo de iteraciones o hasta que se cumpla la tolerancia
    for k in range(max_iter):
        x_new = np.zeros_like(x)
        # Actualizar cada componente de x
        for i in range(n):
            sum_ = np.dot(A[i], x) - A[i, i] * x[i]  # Sumar usando dot product
            x_new[i] = (b[i] - sum_) / A[i, i]

        # Revisar el criterio de convergencia
        if np.linalg.norm(x_new - x, ord=np.inf) < tol:
            print(f"Convergió en {k+1} iteraciones.")
            return x_new
        
        x = x_new  # Actualizar x para la siguiente iteración

    print("Máximo de iteraciones.")
    return x



### (b) Algoritmo de Gauss-Seidel

In [206]:
def gauss_seidel(A, b, x0=None, tol=1e-10, max_iter=100):
    """
    Resuelve el sistema Ax = b usando el método iterativo de Gauss-Seidel.

    Parámetros:
    A (numpy.ndarray): Matriz de coeficientes de tamaño n x n.
    b (numpy.ndarray): Vector de términos independientes.
    x0 (numpy.ndarray): Vector inicial de la solución, por defecto es un vector de ceros.
    tol (float): Tolerancia para el criterio de convergencia.
    max_iter (int): Número máximo de iteraciones.

    Retorna:
    numpy.ndarray: Aproximación de la solución x.
    """
    n = A.shape[0]
    # Inicializar con ceros si no se proporciona un vector inicial
    if x0 is None:
        x0 = np.zeros_like(b)
    x = x0.copy()

    # Iterar hasta el número máximo de iteraciones o hasta que se cumpla la tolerancia
    for k in range(max_iter):
        x_new = x.copy()  # Copiar el valor actual de x
        # Actualizar cada componente de x
        for i in range(n):
            # Sumar los elementos necesarios para el cálculo
            sum_1 = np.dot(A[i, :i], x_new[:i])  # Suma de elementos anteriores (usando x_new)
            sum_2 = np.dot(A[i, i+1:], x[i+1:])  # Suma de elementos posteriores (usando x)

            # Actualizar el valor de x_new
            x_new[i] = (b[i] - sum_1 - sum_2) / A[i, i]

        # Revisar el criterio de convergencia
        if np.linalg.norm(x_new - x, ord=np.inf) < tol:
            print(f"Convergió en {k+1} iteraciones.")
            return x_new
        
        x = x_new  # Actualizar x para la siguiente iteración

    print("Máximo de iteraciones.")
    return x



### (c) Generar matriz A de forma aleatoria y factorizar

In [207]:
A = Mat_Jacobi(100)
# Realizar la factorización QR
Q, R = np.linalg.qr(A)
print("Matriz Q de la factorización QR:")
print(Q)
A_ = Q @ R
print(np.allclose(A, A_))


### (d) Definir matriz diagonal D

In [208]:
n = 100
# Da una secuencia decreciente con el primer valor menor que 1
diagonal_values = np.linspace(0.9, 0.1, n)  # Decreciente desde 0.9 hasta 0.1
D = np.diag(diagonal_values)
print("Matriz diagonal D:")
print(D)


### (e) Definir la nueva matriz \(M = QDQ^T\)

In [209]:
M = Q @ D @ Q.T
# Mostrar la matriz M
print("Matriz M = QDQ^T:")
print(M)

### (f) Generar vector \(x\) y \(b\)

In [210]:
x = np.arange(1, 101)
b =M@ x
print(b)


### (g) Calcular error relativo entre la solución teórica y la aproximada


In [211]:
def error_relativo(x_teorico, x_aproximado):
    """
    Calcula el error relativo entre la solución teórica y la solución aproximada.
    Parámetros:
    x_teorico (numpy.ndarray): Vector solución teórica.
    x_aproximado (numpy.ndarray): Vector solución calculada.
    Retorna:
    float: Error relativo.
    """
    error_absoluto = np.linalg.norm(x_teorico - x_aproximado)
    error_relativo = error_absoluto / np.linalg.norm(x_teorico) if np.linalg.norm(x_teorico) != 0 else np.nan
    return error_relativo

# Generar matriz y vector b
n = 100  # Tamaño de la matriz
M = Mat_Jacobi(n)  # Generar matriz diagonal dominante
x_teorico = np.arange(1, n + 1, dtype=np.float64)  # Vector solución teórica
b = M @ x_teorico  # Calcular el vector b

# Valores iniciales y parámetros
x0 = np.zeros_like(x_teorico)  # Vector inicial
tol = 1e-10  # Tolerancia
max_iter = 100  # Máximo de iteraciones

# Aproximaciones usando Jacobi y Gauss-Seidel
x_aproximado_jacobi = jacobi(M, b, x0, tol, max_iter)
x_aproximado_gauss_seidel = gauss_seidel(M, b, x0, tol, max_iter)

# Calcular errores relativos
err_rel_jacobi = error_relativo(x_teorico, x_aproximado_jacobi)
print(f"Error relativo (Jacobi): {err_rel_jacobi:.6e}")  # Formato científico

# Corregir la línea de Gauss-Seidel (eliminar el "err" adicional)
err_rel_gaussSeidel = error_relativo(x_teorico, x_aproximado_gauss_seidel)
print(f"Error relativo (Gauss-Seidel): {err_rel_gaussSeidel:.6e}")  # Formato científico



### (h) Calcular errores relativos para 30 ecuaciones distintas



In [212]:
# Inicializar listas para almacenar los errores
errores_rel_jacobi = []
errores_rel_gauss_seidel = []

# Definir el número de iteraciones
num_iteraciones = 30

# Vector teórico
x_teorico = np.arange(1, 101, dtype=np.float64)

# Valores iniciales y parámetros
x0 = np.zeros_like(x_teorico)  # Vector inicial
tol = 1e-10  # Tolerancia
max_iter = 100  # Máximo de iteraciones

# Ciclo para las iteraciones
for _ in range(num_iteraciones):
    # Generar matriz ortogonal aleatoria Q
    Q = np.random.rand(100, 100)
    Q, _ = np.linalg.qr(Q)  # Para que Q sea ortogonal

    # Definir la matriz diagonal D con valores decrecientes
    diagonal_values = np.linspace(0.9, 0.1, 100)
    D = np.diag(diagonal_values)

    # Definir la matriz M = Q * D * Q^T
    M = Q @ D @ Q.T

    # Calcular b = M @ x_teorico
    b = M @ x_teorico

    # Resolver con Jacobi y Gauss-Seidel
    x_aprox_jacobi = jacobi(M, b, x0, tol, max_iter)
    x_aprox_gauss_seidel = gauss_seidel(M, b, x0, tol, max_iter)

    # Calcular errores relativos
    error_rel_jacobi = error_relativo(x_teorico, x_aprox_jacobi)
    error_rel_gauss_seidel = error_relativo(x_teorico, x_aprox_gauss_seidel)

    # Guardar los errores en las listas
    errores_rel_jacobi.append(error_rel_jacobi)
    errores_rel_gauss_seidel.append(error_rel_gauss_seidel)

# Convertir las listas de errores a arrays
errores_rel_jacobi = np.array(errores_rel_jacobi)
errores_rel_gauss_seidel = np.array(errores_rel_gauss_seidel)

# Imprimir los resultados
print("Errores relativos (Jacobi):", errores_rel_jacobi)
print("Errores relativos (Gauss-Seidel):", errores_rel_gauss_seidel)



### (i) Graficar errores y justificar la preferencia de un algoritmo


In [213]:
iteraciones = np.arange(1, 31)  # Crear un array de iteraciones del 1 al 30

# Configurar el gráfico
plt.figure(figsize=(10, 6))

# Graficar errores relativos de Jacobi
plt.plot(iteraciones, errores_rel_jacobi, label='Error relativo Jacobi', marker='o', linestyle='-', color='blue')

# Graficar errores relativos de Gauss-Seidel
plt.plot(iteraciones, errores_rel_gauss_seidel, label='Error relativo Gauss-Seidel', marker='x', linestyle='-', color='orange')

# Título y etiquetas
plt.title('Errores Relativos en Jacobi y Gauss-Seidel')
plt.xlabel('Iteración')
plt.ylabel('Error Relativo')

# Configurar escala logarítmica en el eje y
plt.yscale('log')

# Leyenda
plt.legend()

# Ajustar el diseño
plt.tight_layout()

# Mostrar el gráfico
plt.show()




### Justificación de la preferencia del algoritmo

Al observar la gráfica, puedes justificar cuál algoritmo prefieres basándote en los errores relativos. Si uno de los métodos muestra consistentemente errores más bajos que el otro, entonces sería preferible ese método. Sin embargo, si ambos métodos tienen un comportamiento similar, podrías optar por el que consideres más fácil de implementar o entender.



# 3. QR


## (a) Función que obtiene la factorización QR mediante el algoritmo de Gram-Schmidt

In [214]:

def qr_fact(A: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
    """
    Factorización QR de una matriz A utilizando el método de Gram-Schmidt.

    Parámetros:
    A (numpy.ndarray): Matriz cuadrada de tamaño n x n.

    Retorna:
    tuple: (Q, R), donde:
        Q (numpy.ndarray): Matriz ortogonal de tamaño n x n.
        R (numpy.ndarray): Matriz triangular superior de tamaño n x n.

    Excepciones:
    Raises ValueError si la matriz A no es cuadrada o tiene columnas linealmente dependientes.
    """
    m, n = A.shape
    if m != n:
        raise ValueError("La matriz A debe ser cuadrada (n x n).")

    Q = np.copy(A)
    R = np.zeros((n, n))

    for i in range(n):
        R[i, i] = np.linalg.norm(Q[:, i])
        if R[i, i] == 0:
            raise ValueError("La matriz A tiene columnas linealmente dependientes.")
        
        Q[:, i] /= R[i, i]
        
        for j in range(i + 1, n):
            R[i, j] = np.dot(Q[:, i], Q[:, j])
            Q[:, j] -= R[i, j] * Q[:, i]

    return Q, R



## (b) Función que resuelve Rx = Q^Tb


## (c) Definición de A, x y b, y encontrar la solución Ax = b

In [215]:
# Fijar la semilla para la reproducibilidad
np.random.seed(50)

# Generación de una matriz aleatoria A de dimensión 100x100
A = np.random.rand(100, 100)  
# Vector de solución real
x_real = np.arange(1, 101, dtype=np.float64)
# Calcular el vector b como b = A * x_real
b = A @ x_real  

def solve_qr(A: np.ndarray, b: np.ndarray) -> np.ndarray:
    """
    Resuelve el sistema Ax = b utilizando la factorización QR.

    Parámetros:
    A (numpy.ndarray): Matriz de coeficientes de tamaño m x n.
    b (numpy.ndarray): Vector de términos independientes de tamaño m.

    Retorna:
    numpy.ndarray: Aproximación de la solución x.
    """
    Q, R = np.linalg.qr(A)  # Factorización QR de A
    y = Q.T @ b  # Resolvemos Q^T * b = y
    x = np.linalg.solve(R, y)  # Resolvemos R * x = y
    return x

# Solucionador QR
x_calculado = solve_qr(A, b)

# Mostrar la solución calculada
print("Solución calculada x:")
print(x_calculado)

# Reconstruir b usando la solución calculada
b_reconstruido = A @ x_calculado

# Verificar si la reconstrucción es aproximadamente igual a b
print("\n¿Aproximadamente A * x == b?", np.allclose(b, b_reconstruido))

# Generación de otra matriz A escalada
scale = 5.0  # Factor de escala
A_scaled = scale * np.random.rand(100, 100)  

# Definición de x_s como un vector de 1 a 100
x_s = np.arange(1, 101, dtype=np.float64)  

# Calcular b usando la nueva matriz A escalada
b_scaled = A_scaled @ x_s

# Mostrar la solución exacta
print("\nSolución exacta x_s:")
print(x_s)



## (d) Calcular el error relativo para 30 ecuaciones

In [216]:
def calcular_error_relativo(x_teorico: np.ndarray, x_aproximado: np.ndarray) -> float:
    """
    Calcula el error relativo entre la solución teórica y la solución aproximada.

    Parámetros:
    x_teorico (numpy.ndarray): Vector solución teórica.
    x_aproximado (numpy.ndarray): Vector solución calculada.

    Retorna:
    float: Error relativo.
    """
    error_absoluto = np.linalg.norm(x_teorico - x_aproximado)
    error_relativo = error_absoluto / np.linalg.norm(x_teorico)
    return error_relativo

# Inicializar lista para almacenar los errores relativos
errores_relativos_qr = []

# Ejecutar 30 iteraciones
for _ in range(30):
    # Generar una matriz A aleatoria de tamaño 100x100
    A = np.random.rand(100, 100)
    # Vector teórico
    x_real = np.arange(1, 101, dtype=np.float64)
    # Calcular b como b = A @ x_real
    b = A @ x_real  

    # Resolver el sistema para x usando factorización QR
    x_calculado = solve_qr(A, b)

    # Calcular el error relativo
    error_rel = calcular_error_relativo(x_real, x_calculado)
    errores_relativos_qr.append(error_rel)

# Convertir la lista de errores relativos a un array de NumPy
errores_rel_qr = np.array(errores_relativos_qr)

# Mostrar los errores relativos de las 30 iteraciones
print("Errores relativos de las 30 iteraciones:")
print(errores_rel_qr)


# 4. Comparacion 

In [219]:
import numpy as np
import matplotlib.pyplot as plt

# Asumiendo que ya tienes los errores relativos de cada método calculados correctamente:
metodos = {
    'Jacobi': errores_rel_jacobi,
    'Gauss-Seidel': errores_rel_gauss_seidel,
    'Cholesky': errores_rel_cholesky,
    'QR': errores_relativos_qr,
    'PLU': errores_rel_plu
}

# Inicializar listas para las medias y desviaciones
medias = []
desviaciones = []

# Calcular media y desviación estándar para cada método
for metodo, errores in metodos.items():
    media_error = np.mean(errores)
    desviacion_error = np.std(errores)
    medias.append(media_error)
    desviaciones.append(desviacion_error)

    # Imprimir la media de los errores en el formato adecuado
    print(f'{metodo.ljust(15)}: {media_error:.16e}')

# Crear una figura para graficar los errores relativos
plt.figure(figsize=(12, 7))

# Graficar los errores relativos de cada método
for metodo, errores in metodos.items():
    plt.plot(errores, label=f'Error relativo {metodo}', marker='o')

plt.title('Comparación de Errores Relativos de Diferentes Métodos')
plt.xlabel('Número de Iteración')
plt.ylabel('Error Relativo')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.yscale('log')
plt.show()

# Graficar medias y desviaciones estándar
plt.figure(figsize=(12, 7))
plt.errorbar(list(metodos.keys()), medias, yerr=desviaciones, fmt='o', capsize=5, 
             color='orange', ecolor='red', elinewidth=2)
plt.title('Media y Desviación Estándar de los Errores Relativos')
plt.ylabel('Error Relativo')
plt.xlabel('Métodos')
plt.grid(True)
plt.tight_layout()
plt.yscale('log')
plt.show()



### (a) ¿Cuál es el solucionador más preciso?

El solucionador **más preciso** es aquel que tiene el menor error relativo en todas las iteraciones.  
Según la gráfica, el **método QR** tiene el error relativo más bajo, fluctuando entre \(10^{-12}\) y \(10^{-13}\).  
A pesar de algunas fluctuaciones, su comportamiento general es más estable en valores pequeños que el de los otros métodos, lo que lo convierte en el más preciso.

---

### (b) ¿Cuál es el solucionador más exacto?

La **exactitud** se refiere a qué tan cerca está una solución del valor real.  
En este caso, también se puede decir que el método **QR** es el más exacto. Dado que el error relativo se mantiene muy bajo y consistente, podemos confiar en que este método produce soluciones muy cercanas a las verdaderas, especialmente en comparación con métodos como PLU o Cholesky que presentan errores mayores.

---

### (c) Si la matriz A es una matriz cuadrada e invertible, ¿Cuál método utilizarías para resolver la ecuación Ax = b?

Para una matriz cuadrada e invertible, la elección del método depende de la precisión que busques y del tipo de matriz.  
**Método sugerido:** Usaría el método **QR**, ya que ofrece la mayor precisión, como se observa en la gráfica.  
Sin embargo, si la matriz tiene ciertas propiedades (como simetría o ser definida positiva), podría ser conveniente considerar otros métodos.  
Por ejemplo, si la matriz es simétrica y definida positiva, el método de **Cholesky** podría ser más eficiente en términos computacionales.

---

### (d) Si hay un método que soluciona la ecuación para una matriz invertible, ¿Cuáles son las ventajas de los otros algoritmos frente al solucionador QR?

Aunque el método **QR** es muy preciso y exacto, los otros métodos tienen ventajas en ciertos escenarios:

1. **PLU**:  
   - Ventaja: Es eficiente para resolver sistemas de ecuaciones cuando necesitas resolver múltiples sistemas \( Ax = b \) con la misma matriz \( A \) pero diferentes \( b \).  
   - Es más rápido que QR en algunos casos y es adecuado para matrices generales.

2. **Cholesky**:  
   - Ventaja: Este método es extremadamente eficiente para matrices que son simétricas y definidas positivas.  
   - Si se sabe que la matriz \( A \) tiene estas propiedades, Cholesky es más rápido que QR o PLU.

3. **Jacobi y Gauss-Seidel (métodos iterativos)**:  
   - Ventaja: Son útiles cuando se trabajan con matrices muy grandes o dispersas, donde los métodos directos (QR, PLU, Cholesky) serían demasiado costosos computacionalmente.  
   - Aunque no son tan precisos como QR, estos métodos pueden ser suficientes en muchos casos prácticos y pueden reducir considerablemente el costo computacional cuando se busca una solución aproximada.

---

